<a href="https://colab.research.google.com/github/napatswift/mtr-en2th/blob/main/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.2 MB/s eta 0:00:00


In [2]:
# configuration

train_sentence_piece = False

In [3]:
import pandas as pd
import sentencepiece as spm # tokenizer

ใช้ colab authentication api เพื่อจะดึงข้อมูลจากชีต

In [4]:
from google.colab import auth
auth.authenticate_user()

In [5]:
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [6]:
spread_sheet = gc.open_by_key('1wd-mlGq_XETPD6szP-Q9XJ9xl8SVOMhmakpQLEkkSrg')
sheet = spread_sheet.worksheets()[0] # เลือกแผ่นแรก
dataset_df = pd.DataFrame(sheet.get_all_records())

In [7]:
if train_sentence_piece:
  !touch lines.txt

In [8]:
# create dummy function to prevent polluting global namespace
def _f():
  sentence_lines = dataset_df[(dataset_df.Thai != '<song title>') &
           (~(dataset_df.Thai == '')) &
           (~(dataset_df.English == ''))][['Thai', 'English']].values
  total_line_count = 0
  with open('lines.txt', 'w') as f:
    string_lines = [str(m) for l in sentence_lines for m in l]
    for l in string_lines:
      f.write(l)
      f.write('\n')
      total_line_count += 1
    print('total_line_count', total_line_count)

if train_sentence_piece:
  _f()

create folder to store sentence piece model

In [9]:
!if [[ ! -e model ]];\
  then mkdir model;\
fi

In [10]:
if train_sentence_piece:
  spm.SentencePieceTrainer.train(
        input='lines.txt',
        model_prefix='model/m',
        vocab_size=4_800,
        pad_id=3)
  model = spm.SentencePieceProcessor(model_file='model/m.model')
  print(*model.encode("""ชีวิตของผมสั้นไป แต่ผมก็มีชีวิตที่ดีมาก ๆ นะ
My life was kinda short, but I got so many blessings
ผมมีความสุขมากที่คุณเคยเป็นคนรักของผม มันแค่ห่วยมากตรงที่เราต้องจบกันแบบนี้
Happy you were mine, it sucks that it’s all ending""".split('\n'), out_type=str), sep='\n')

โหลด sentencepiece จาก github

In [15]:
!git clone https://github.com/napatswift/mtr-en2th.git

Cloning into 'mtr-en2th'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 2), reused 13 (delta 2), pack-reused 0
Unpacking objects: 100% (13/13), 281.55 KiB | 1.26 MiB/s, done.


In [16]:
class TokenizerWraperForSP:
  def __init__(self, spm):
    self.spm = spm
    self.vocab_size = spm.vocab_size()
    self.pad_id = spm.pad_id()
  
  def tokenize(self, text_or_list, **kargs):
    return self.spm.encode(text_or_list, **kargs)

In [19]:
tokenizer = TokenizerWraperForSP(
    spm=spm.SentencePieceProcessor(model_file='mtr-en2th/spmodel/m48.model')
)

# Dataset

เลือกแถวที่จับคู่แล้ว `dataset_df['Matched?'] == 'TRUE'` และ ไม่ใช่แถวที่เป็นชื่อเพลง `dataset_df.Thai != '<song title>'`

In [32]:
import tensorflow as tf
import numpy as np

In [20]:
en_th_ds = dataset_df[(dataset_df['Matched?'] == 'TRUE') & (dataset_df.Thai != '<song title>')][['English','Thai',]]

In [21]:
en_th_ds.loc[:, 'en2id'] = tokenizer.tokenize(en_th_ds.English.astype(str).to_list(), add_bos=True, add_eos=True,)
en_th_ds.loc[:, 'th2id'] = tokenizer.tokenize(en_th_ds.Thai.astype(str).to_list(), add_bos=True, add_eos=True,)

In [22]:
seq_max_len = en_th_ds.en2id.apply(len).max()

In [125]:
sequences = en_th_ds.en2id.apply(lambda seq: seq + [tokenizer.pad_id]*(seq_max_len-len(seq))).to_list()
x_train = np.array(sequences)
y_train = en_th_ds.th2id.apply(lambda seq: seq + [tokenizer.pad_id]*(seq_max_len-len(seq))).to_list()
# y_train = np.array([tf.keras.utils.to_categorical(y, num_classes=tokenizer.vocab_size).sum(0) for y in y_train])

# Embeddings

In [131]:
def gen_data(xs, ys):
  for x, y in zip(xs, ys):
    for yi in y:
      yield x, yi

ds = tf.data.Dataset.from_generator(
    lambda: gen_data(x_train, y_train),
    output_types=(tf.int16, tf.int16))

In [132]:
next(iter(ds))

(<tf.Tensor: shape=(54,), dtype=int16, numpy=
 array([   1,  471,    7,    6,   37,   11, 2084,    2,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3,    3,
           3,    3,    3,    3,    3,    3,    3,    3,    3,    3],
       dtype=int16)>, <tf.Tensor: shape=(), dtype=int16, numpy=1>)

In [105]:
input = tf.keras.layers.Input((54,))
y = tf.keras.layers.Embedding(tokenizer.spm.vocab_size(), 512,)(input)
y = tf.keras.layers.Dropout(0.5)(y)
y = tf.keras.layers.Flatten()(y)
y = tf.keras.layers.Dropout(0.5)(y)
y = tf.keras.layers.Dense(1024, 'relu')(y)
y = tf.keras.layers.Dropout(0.5)(y)
y = tf.keras.layers.Dense(tokenizer.spm.vocab_size(), 'softmax')(y)
model = tf.keras.models.Model(inputs=input,outputs=y)

In [114]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanAbsoluteError(),
              metrics=['accuracy'])

In [115]:
model.fit(x_train[:100], y_train[:100],
          batch_size=12,
          validation_data=(x_train[-100:], y_train[-100:]))

9/9 [==============================] - 7s 648ms/step - loss: 0.0110 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000


In [116]:
model.predict(x_train[:1])

1/1 [==============================] - 0s 80ms/step


array([[1.1019687e-32, 2.9411009e-17, 1.5045997e-16, ..., 6.4621457e-33,
        5.1085639e-34, 1.0616384e-32]], dtype=float32)

In [1]:
import pandas as pd

In [3]:
ds_df = pd.read_csv('dataset/translate_test.csv')

array([['oh, even the white kids!', 'โอ้ แม้แต่เจ้าเด็กผิวขาวก็ด้วย'],
       ['lay on the floor of your living room',
        'มาลงจอดบนพื้นห้องนั่งเล่นของเธอ'],
       ['and i’m pretendin’ you ain’t been on my mind',
        'และฉันแกล้งทำเป็นว่าไม่มีเธออยู่ในใจ'],
       ...,
       ['i see the way you vibin’, keep me hypnotized and',
        'ฉันเห็นท่าทางของเธอ ที่คอยสะกดจิตฉันและ'],
       ['stay cool, it’s just a kiss', 'ใจเย็นน่า มันก็แค่จูบ จุ๊บ ๆ'],
       ['hair toss, check my nails', 'สะบัดหัว เช็คดูเล็บ']], dtype=object)

In [5]:
ds_df.values[:10]

array([['oh, even the white kids!', 'โอ้ แม้แต่เจ้าเด็กผิวขาวก็ด้วย'],
       ['lay on the floor of your living room',
        'มาลงจอดบนพื้นห้องนั่งเล่นของเธอ'],
       ['and i’m pretendin’ you ain’t been on my mind',
        'และฉันแกล้งทำเป็นว่าไม่มีเธออยู่ในใจ'],
       ['‘cause i warned you, ooh, ooh, ooh, ooh',
        'เพราะฉันเตือนเธอแล้ว'],
       ['so many things to say if you stay',
        'ฉันมีหลายสิ่งจะพูดไป ถ้าเธอยังอยู่ต่อนะ'],
       ['you never told me', 'เธอไม่เคยเอ่ยบอกกัน'],
       ['you’re not alone like you think you are',
        'คุณไม่ได้โดดเดี่ยวเพียงคนเดียวอย่างที่คุณคิด'],
       ['lies on lies on lies and every single time you call, i fall for this',
        'โกหกซ้ำไปซ้อนมา และทุกครั้งที่เธอโทรหากัน ฉันก็ใจอ่อนทุกครั้ง'],
       ['but everyday seems like saturday with you',
        'แต่ไม่ว่าวันไหนก็เหมือนวันเสาร์ เมื่อฉันได้อยู่กับเธอ'],
       ['bet your house and i’ll call it',
        'วางเดิมพันทุกอย่างเลยสิ เดี๋ยวผมจะชนะเดิมพันนั้นเองแหละ']],
     